<a href="https://colab.research.google.com/github/Bingurrr/autonomous_vehicle/blob/main/%EB%A1%9C%EB%B3%B4%ED%83%9D%EC%8B%9C%EC%8A%B9%EA%B0%9D%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe

     |████████████████████████████████| 47.4MB 114kB/s 


Mediapipe holistic
동작을 파악한다.

In [2]:
#pose인식

import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic


def holisticVideo(video_path) :
  frames = []
  cap = cv2.VideoCapture(video_path)
  ret, frame = cap.read()
  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
      ret, frame = cap.read()
      if not ret : break
      # Flip the image horizontally for a later selfie-view display, and convert
      # the BGR image to RGB.
      image = cv2.flip(frame, 1)
      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      results = holistic.process(image)

      # Draw landmark annotation on the image.
      image.flags.writeable = True
      #face
      mp_drawing.draw_landmarks(
          image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
      #left hand
      mp_drawing.draw_landmarks(
          image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
      #right hand
      mp_drawing.draw_landmarks(
          image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
      #pose
      mp_drawing.draw_landmarks(
          image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
      #cv2_imshow(image)

      frames.append(image)

      if cv2.waitKey(5) & 0xFF == 27:
        break


    fps = 30
    height, width, layers = frames[0].shape
    size = (width, height)
    out = cv2.VideoWriter(video_path,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frames)) :
      out.write(frames[i])
    out.release()

  cap.release()

In [3]:
video_path = "/content/drive/MyDrive/최종과제/틱톡.mp4" # input your video path
holisticVideo(video_path)

In [4]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [5]:
import csv
import os
import numpy as np

In [22]:
  frames = []
  video_path = "/content/drive/MyDrive/최종과제/틱톡.mp4"
  cap = cv2.VideoCapture(video_path)
  ret, frame = cap.read()
  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
      ret, frame = cap.read()
      if not ret : break
      # Flip the image horizontally for a later selfie-view display, and convert
      # the BGR image to RGB.
      image = cv2.flip(frame, 1)
      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      results = holistic.process(image)

      # Draw landmark annotation on the image.
      image.flags.writeable = True
      #face
      mp_drawing.draw_landmarks(
          image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
      #left hand
      mp_drawing.draw_landmarks(
          image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
      #right hand
      mp_drawing.draw_landmarks(
          image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
      #pose
      mp_drawing.draw_landmarks(
          image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
      frames.append(image)
    
      #Export coordinates
      try :
        #extract pose landmark
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                             for landmark in pose]).flatten())
        #concate rows   
        row = pose_row

        #append class name
        row.insert(0,class_name)

        #export to csv
        with open('coor', mode='a', newline='') as f:
          csv_writer = csv.writer(f, delimiter =',', quotechar='"', quoting = csv.QUOTE_MINIMAL)
          csv_writer.writerow(row)

      except :
        pass

In [24]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [27]:
landmarks = ['class']
for val in range(1, num_coords+1):
  landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [28]:
with open('coords.csv', mode = 'w', newline ='' ) as f:
  csv_writer = csv.writer(f, delimiter=',', quotechar ='"', quoting = csv.QUOTE_MINIMAL)
  csv_writer.writerow(landmarks)

In [35]:
class_name ="TAXI"

Read in Collected Data and Process

택시를 잡는 모습은 TAXI라고 하였다.
그 외의 모습은 NOTHING라고 표현 하였다.
TAXI를 잡는 모습의 데이터는 내가 택시를 잡는 모습의 영상들을 촬영하여 데이터를 넣어 주었고 NOTHING의 데이터 들은 유투브등 사람들의 일반적인 모습들의 영상을 넣어 주었다.


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split

coords.csv파일은 깃에 있습니다.

In [36]:
df = pd.read_csv('coords.csv')

In [37]:
df[df['class']=='Taxi']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,z3,v3,x4,y4,z4,v4,x5,y5,z5,v5,x6,y6,z6,v6,x7,y7,z7,v7,x8,y8,z8,v8,x9,y9,z9,v9,x10,y10,z10,...,x24,y24,z24,v24,x25,y25,z25,v25,x26,y26,z26,v26,x27,y27,z27,v27,x28,y28,z28,v28,x29,y29,z29,v29,x30,y30,z30,v30,x31,y31,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Taxi,0.458574,0.227861,-0.599831,1.0,0.457035,0.190570,-0.571932,1.0,0.461865,0.191442,-0.572013,0.994140,0.466917,0.192178,-0.572094,1.0,0.438702,0.186296,-0.598259,1.0,0.429870,0.183949,-0.598316,0.994772,0.421059,0.181782,-0.598343,1.0,0.451770,0.198833,-0.409417,0.999014,0.388048,0.187889,-0.525103,0.999089,0.456336,0.263126,-0.534690,...,0.427769,0.827682,0.020475,0.998909,0.320065,0.846058,-0.018927,0.998173,0.431562,1.109236,0.204462,0.308679,0.313975,1.121859,0.145166,0.242922,0.405482,1.317071,0.555231,0.008589,0.316142,1.315046,0.545281,0.003802,0.397036,1.340645,0.585361,0.020228,0.309283,1.336810,0.575742,0.015263,0.415907,1.418157,0.489810,0.003964,0.345671,1.415789,0.472169,0.001296
1,Taxi,0.457867,0.247191,-0.569157,1.0,0.456546,0.210656,-0.545905,1.0,0.461321,0.211858,-0.545994,0.622890,0.466333,0.212881,-0.546095,1.0,0.439220,0.204826,-0.572892,1.0,0.431063,0.201440,-0.572930,0.572806,0.423010,0.198284,-0.573064,1.0,0.452096,0.218129,-0.396788,0.908379,0.391694,0.201177,-0.519490,0.871764,0.454598,0.281706,-0.508492,...,0.430185,0.846753,0.034670,0.991451,0.324072,0.870676,-0.032929,0.980987,0.437467,1.101570,0.204393,0.018330,0.323162,1.120400,0.109335,0.019941,0.412739,1.298223,0.544956,0.000991,0.321350,1.306621,0.505103,0.000655,0.401826,1.322590,0.574536,0.004190,0.314761,1.327408,0.535726,0.006277,0.432310,1.396490,0.489183,0.000507,0.347087,1.407731,0.441442,0.000260
2,Taxi,0.453699,0.240966,-0.533934,1.0,0.454153,0.208361,-0.505721,1.0,0.459135,0.211569,-0.505771,0.874007,0.464329,0.211448,-0.505833,1.0,0.436871,0.197994,-0.534885,1.0,0.428437,0.193228,-0.534980,0.896416,0.420043,0.188806,-0.535176,1.0,0.450570,0.216318,-0.341142,0.970771,0.388486,0.191869,-0.474639,0.969105,0.451106,0.277248,-0.468339,...,0.428390,0.866474,0.042669,0.994230,0.323552,0.890697,-0.041009,0.991770,0.434231,1.132331,0.191729,0.026806,0.322467,1.166316,0.055558,0.040000,0.408300,1.384963,0.496185,0.001709,0.306318,1.393234,0.385673,0.001080,0.395204,1.426773,0.519418,0.007089,0.294926,1.427393,0.408018,0.009411,0.432885,1.475912,0.417510,0.000832,0.332698,1.491477,0.289427,0.000348
3,Taxi,0.453138,0.233564,-0.576956,1.0,0.452202,0.204198,-0.544489,1.0,0.456402,0.208155,-0.544448,0.989857,0.460822,0.209672,-0.544456,1.0,0.436406,0.192611,-0.576063,1.0,0.428485,0.187789,-0.576107,0.993490,0.420572,0.183341,-0.576248,1.0,0.446334,0.215493,-0.362833,0.998419,0.389619,0.186935,-0.504051,0.997751,0.449788,0.270657,-0.505764,...,0.429074,0.884609,0.051060,0.994581,0.322593,0.901193,-0.049579,0.994678,0.438113,1.175756,0.193205,0.135623,0.308692,1.196590,0.074201,0.176069,0.405774,1.441068,0.533708,0.007022,0.296339,1.443220,0.446987,0.002108,0.390234,1.480225,0.560662,0.021734,0.284429,1.479510,0.472556,0.009348,0.435102,1.538431,0.454248,0.003889,0.328516,1.540529,0.341509,0.000690
4,Taxi,0.453449,0.223263,-0.573246,1.0,0.451522,0.193305,-0.536313,1.0,0.455486,0.196332,-0.536321,0.965452,0.459637,0.198324,-0.536273,1.0,0.435978,0.183673,-0.570805,1.0,0.428163,0.179643,-0.570763,0.973708,0.420362,0.175921,-0.570862,1.0,0.445408,0.204972,-0.345548,0.996055,0.389451,0.180708,-0.496666,0.991610,0.450608,0.260037,-0.499170,...,0.432693,0.888917,0.050937,0.997418,0.324211,0.913710,-0.049393,0.997571,0.441820,1.207817,0.088772,0.519358,0.310999,1.230121,-0.012655,0.591356,0.414510,1.507890,0.433388,0.021831,0.308489,1.516474,0.344283,0.007130,0.401843,1.549448,0.459594,0.056265,0.299453,1.556017,0.367298,0.018432,0.441815,1.612062,0.343851,0.009726,0.335934,1.619215,0.221486,0.001821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [38]:
df[df['class']=='Nothing']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,z3,v3,x4,y4,z4,v4,x5,y5,z5,v5,x6,y6,z6,v6,x7,y7,z7,v7,x8,y8,z8,v8,x9,y9,z9,v9,x10,y10,z10,...,x24,y24,z24,v24,x25,y25,z25,v25,x26,y26,z26,v26,x27,y27,z27,v27,x28,y28,z28,v28,x29,y29,z29,v29,x30,y30,z30,v30,x31,y31,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
188,Nothing,0.564520,0.335978,0.024870,1.0,0.559633,0.325529,0.012507,1.0,0.558608,0.325483,0.012600,0.997368,0.557661,0.325432,0.012565,1.0,0.563378,0.325334,0.012164,1.0,0.564511,0.325444,0.012222,0.997546,0.565658,0.325783,0.012227,1.0,0.556379,0.335311,-0.000435,0.999087,0.569488,0.334502,-0.002240,0.994027,0.565878,0.347884,0.025133,...,0.608075,0.565611,0.017012,0.999051,0.634331,0.556963,-0.016815,0.993741,0.642328,0.511453,-0.106756,0.461952,0.623761,0.527509,-0.112480,0.279191,0.653351,0.724479,-0.248659,0.756641,0.662433,0.726940,-0.267224,0.424743,0.651305,0.748718,-0.261932,0.929226,0.665112,0.755664,-0.281434,0.758103,0.659552,0.795262,-0.298301,0.743197,0.688175,0.777154,-0.296217,0.505513
189,Nothing,0.559072,0.332729,-0.049773,1.0,0.555576,0.326553,-0.066254,1.0,0.553568,0.327801,-0.066074,0.999997,0.551600,0.328990,-0.066059,1.0,0.561993,0.324856,-0.067342,1.0,0.564148,0.324937,-0.067221,0.999991,0.566264,0.325222,-0.067297,1.0,0.550680,0.344850,-0.091217,0.999996,0.572300,0.338345,-0.096248,0.999916,0.558911,0.347236,-0.053009,...,0.582101,0.568046,0.019631,0.999951,0.625848,0.536541,-0.019595,0.999753,0.598548,0.628198,-0.004541,0.762055,0.668639,0.517181,-0.020219,0.870966,0.611055,0.803553,-0.109774,0.908132,0.673451,0.688006,-0.157500,0.863759,0.610762,0.829585,-0.120068,0.962849,0.668033,0.718595,-0.171279,0.936729,0.620493,0.851152,-0.131605,0.643868,0.702245,0.722763,-0.148364,0.724574
190,Nothing,0.526627,0.358669,-0.030834,1.0,0.522865,0.351204,-0.047820,1.0,0.521604,0.351571,-0.047843,0.999586,0.520387,0.351799,-0.047947,1.0,0.527637,0.349094,-0.035592,1.0,0.529493,0.348077,-0.035506,0.999545,0.531361,0.347235,-0.035547,1.0,0.522168,0.356180,-0.098821,0.999339,0.538008,0.350962,-0.040403,0.997179,0.527911,0.368613,-0.046342,...,0.563488,0.536182,-0.033717,0.991302,0.590425,0.525637,0.033910,0.993442,0.516159,0.561020,-0.013964,0.067649,0.587229,0.610289,0.056656,0.220373,0.488142,0.718475,-0.033822,0.119400,0.545877,0.743458,0.016170,0.381052,0.488798,0.726614,-0.037331,0.202181,0.547312,0.768039,0.010743,0.522866,0.467939,0.672370,-0.063789,0.040823,0.530456,0.781297,-0.018292,0.180857
191,Nothing,0.529930,0.347633,-0.126854,1.0,0.526919,0.342989,-0.143491,1.0,0.525050,0.344196,-0.143429,0.999977,0.523210,0.345277,-0.143449,1.0,0.533036,0.340047,-0.140459,1.0,0.535293,0.339192,-0.140346,0.999967,0.537518,0.338614,-0.140340,1.0,0.524843,0.355519,-0.175106,0.999985,0.544582,0.346464,-0.156733,0.999889,0.530052,0.359946,-0.132789,...,0.552701,0.543099,-0.000951,0.999800,0.583284,0.528425,0.001194,0.999438,0.527465,0.577062,0.031437,0.767247,0.566657,0.614367,-0.031455,0.950210,0.496697,0.679090,0.069779,0.722435,0.555968,0.764065,-0.014448,0.961419,0.494581,0.688495,0.072508,0.725399,0.555761,0.785697,-0.014045,0.942061,0.482543,0.693999,0.056939,0.390773,0.550277,0.816951,-0.046022,0.782892
192,Nothing,0.532285,0.338674,-0.160885,1.0,0.528641,0.332185,-0.173275,1.0,0.527228,0.332903,-0.173198,0.999999,0.525878,0.333530,-0.173250,1.0,0.533763,0.330874,-0.173073,1.0,0.535670,0.330657,-0.172985,0.999999,0.537654,0.330678,-0.172909,1.0,0.525894,0.344765,-0.181939,1.000000,0.544492,0.340641,-0.178725,0.999995,0.532761,0.351383,-0.158675,...,0.544614,0.548559,0.023118,0.999978,0.567346,0.539203,-0.022575,0.999968,0.522265,0.599293,0.113955,0.899485,0.560903,0.641919,-0.043558,0.994677,0.471041,0.667969,0.186650,0.897453,0.548747,0.773837,0.019417,0.993708,0.463201,0.675310,0.193072,0.838932,0.546398,0.790935,0.023536,0.962100,0.457294,0.691709,0.168729,0.810364,0.546587,0.826128,-0.026031,0.965200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [39]:
x = df.drop('class', axis = 1)
y = df['class']

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 1234)

Train Machine Learning Classification Model

In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [42]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [43]:
fit_models = {}
for algo, pipeline in pipelines.items():
  model = pipeline.fit(x_train, y_train)
  fit_models[algo] = model

In [44]:
from sklearn.metrics import accuracy_score
import pickle

In [45]:
for algo, model in fit_models.items():
  yhat = model.predict(x_test)
  print(algo, accuracy_score(y_test,yhat))

lr 1.0
rc 1.0
rf 1.0
gb 0.996969696969697


In [46]:
model

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(ccp_alpha=0.0,
                                            criterion='friedman_mse', init=None,
                                            learning_rate=0.1, loss='deviance',
                                            max_depth=3, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100,
                                            n_iter_no_

In [47]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic



택시를 잡는 동작인지 아닌지 확인해주는 동영상
로보택시가 사람을 승객인지 그렇지 않은지 파악

In [48]:
frames = []
video_path = "/content/drive/MyDrive/최종과제/택시최종.mp4"
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    ret, frame = cap.read()
    if not ret : break
    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.flip(frame, 1)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    #face
    mp_drawing.draw_landmarks(
        image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    #left hand
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    #right hand
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    #pose
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    
    try :
      #extract pose landmark
      pose = results.pose_landmarks.landmark
      pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                             for landmark in pose]).flatten())
      #concate rows   
      row = pose_row
      x = pd.DataFrame([row])
      body_language_class = model.predict(x)[0]
      body_language_prob = model.predict_proba(x)[0]
      #print(body_language_class, body_language_prob)
      coords = tuple(np.multiply(
              np.array(
                  (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y,)
                  ), [640,480]).astype(int))
      
      cv2.putText(image, body_language_class, coords,
                  cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
      
      frames.append(image)

    except :
      pass
    
  fps = 30
  height, width, layers = frames[0].shape
  size = (width, height)
  out = cv2.VideoWriter(video_path,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  for i in range(len(frames)) :
    out.write(frames[i])
  out.release()


손모양 인식
손모양 인식을 통해 승객을 보다 더 정확하게 인식 할 수 있다.

In [ ]:
#손 모양인식
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
import numpy as np
import uuid
import os
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands



def handsVideo(video_path) :
  frames = []
  cap = cv2.VideoCapture(video_path)
  ret, frame = cap.read()
  with mp_hands.Hands(min_detection_confidence= 0.8, min_tracking_confidence = 0.5) as hands :
    while cap.isOpened():
      ret, frame = cap.read()
      if not ret : break
      #BGR 2 RGB
      image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
      #set flag
      image.flags.writeable = False
      #Dections
      results = hands.process(image)
      #set flag to true
      image.flags.writeable = True
      #Detections
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      #frames.append(results)
      #rendering results
      if results.multi_hand_landmarks :
        for num, hand in enumerate(results.multi_hand_landmarks):
          mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)

      frames.append(image)
      if cv2.waitKey(10) & 0xFF == ord('q') :  break

    fps = 30
    height, width, layers = frames[0].shape
    size = (width, height)
    out = cv2.VideoWriter(video_path,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frames)) :
      out.write(frames[i])
    out.release()
  

In [ ]:
# Test
video_path = "/content/drive/MyDrive/최종과제/손.mp4" # input your video path
handsVideo(video_path)